In [1]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.compat.v1.enable_eager_execution()

pixel = 128
batch_size = 128
modelname = 'gaby_full_resnet_sample'
data_root_orig = './testData'

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
def testAnImage(filename, label_names, modelGo):
    check = load_and_preprocess_image(filename)
    # check = tf.image.rot90(check, k=1)
    plt.imshow(check)
    plt.show()
    print(type(check))
    predicts = modelGo.predict(np.asarray([check]))
    print(label_names)
    print(predicts)

In [3]:
data_root = pathlib.Path(data_root_orig)
print(data_root)
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
#label_names.append('Others')
print(label_names)
label_to_index = dict((name, i) for i, name in enumerate(label_names))


for item in data_root.iterdir():
    print(item)

all_image_paths = list(data_root.glob('*/*.jpg'))
all_image_paths = sorted(all_image_paths)

#print(all_image_paths)

# Load trained model
filename = modelname + '.h5' # ResNet Model using Feature Wise in imagedatagenerator
ResNetModel = load_model(filename)

tdatagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    width_shift_range=0,
    height_shift_range=0,
    rotation_range=0,
    zoom_range=0,
    # shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

test_set_df = pd.read_csv('./test_GabyNg_set.csv')

test_generator = tdatagen.flow_from_dataframe(dataframe=test_set_df, x_col="filename",
                                                                y_col="label",
                                                                class_mode="categorical",
                                                                target_size=(pixel, pixel),
                                                                shuffle=False,
                                                                batch_size=batch_size)

pred = ResNetModel.predict(test_generator)
predout = np.argmax(pred, axis=1)
testout = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('testout:', testout)
print('predout:', predout)

testScores = metrics.accuracy_score(testout, predout)
confusion = metrics.confusion_matrix(testout, predout)

print("Best accuracy (on testing dataset): %.2f%%" % (testScores * 100))
print(metrics.classification_report(testout, predout, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion)


testData
['GabyNg', 'LeeSeng', 'XiaoYan']
testData/GabyNg
testData/XiaoYan
testData/LeeSeng
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 21 validated image filenames belonging to 3 classes.
testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [0 0 1 1 1 2 1 2 1 1 1 1 0 0 1 0 1 0 1 2 1]
Best accuracy (on testing dataset): 38.10%
              precision    recall  f1-score   support

      GabyNg     0.3333    0.4000    0.3636         5
     LeeSeng     0.3333    0.5714    0.4211         7
     XiaoYan     0.6667    0.2222    0.3333         9

    accuracy                         0.3810        21
   macro avg     0.4444    0.3979    0.3727        21
weighted avg     0.4762    0.3810    0.3698        21

[[2 3 0]
 [2 4 1]
 [2 5 2]]
